# Basic Prompt Engineering (Korean Language)

## Step 1. Prepare Large Language Model (LLM) and Embedding Model 
---

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [4]:
import time
import sagemaker, boto3, json
import glob
import os
import pandas as pd
import requests
import json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

In [5]:
!pip install --upgrade pip --quiet --disable-pip-version-check --root-user-action=ignore

In [6]:
!pip install -qU nltk faiss-cpu langchain wikipedia pdfplumber unstructured

In [7]:
RESTAPI_ID = "p50bg2i2id" # YOUR RESTAPI ID
URL = f'https://{RESTAPI_ID}.execute-api.{aws_region}.amazonaws.com/api/'.replace('"','')
LLM_URL = f"{URL}/llm/kkulm_12_8b"
EMB_URL = f"{URL}/emb/kosimcse"
HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

<br>

## Step 2. Ask a question to LLM without RAG
---

### Simple prompt engineering

In [59]:
from lib_ko import Prompter, get_payload
from langchain.llms import AmazonAPIGateway

params = {
    "do_sample": False,
    "max_new_tokens": 128,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}
llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)
llm.model_kwargs = params

In [60]:
prompter  = Prompter("kullm")

In [62]:
%%time

payload = {
    "inputs": "Generative AI는",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

 인간의 지식을 바탕으로 새로운 것을 창조하는 능력이다. GPT-3와 같은 언어 모델은 텍스트를 생성할 수 있지만, GPT-4와 같은 언어 모델은 텍스트를 생성하고, 생성된 텍스트에 대한 설명을 제공하고, 생성된 텍스트를 기반으로 예측을 할 수 있습니다. 이러한 모델은 인간과 유사한 텍스트를 생성하지만, 인간과 유사한 텍스트를 생성하기 위해 인간의 언어에 의존합니다. 반면에 생성적 AI는 인간의 지식 없이도 텍스트를 생성할 수 있으며, 텍스트를 생성하기
CPU times: user 13.6 ms, sys: 0 ns, total: 13.6 ms
Wall time: 6.51 s


In [64]:
%%time

payload = {
    "inputs": """Amazon SageMaker의 주요 기능을 간략하게 설명하는 이메일 메시지

여러분 안녕하세요.

다음 사항을 발표합니다.""",
    "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False}
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])


Amazon SageMaker는 Amazon Web Services의 새로운 기능으로, AWS에서 제공하는 강력한 머신 러닝 서비스입니다.
이 서비스는 머신 러닝 모델을 구축하고 배포하는 데 필요한 모든 것을 제공
CPU times: user 16.3 ms, sys: 3.29 ms, total: 19.6 ms
Wall time: 3.35 s


### More complex prompts: Play the role of AWS SA


In [78]:
%%time
instruction = """
당신은 AWS에 대한 경험이 풍부한 솔루션 아키텍트입니다. 고객 요구 사항을 분석하여 잘 설계된 솔루션 아키텍처를 만들어 고객에게 제시합니다. 
아키텍처는 상세하고 친절해야 합니다. 다음과 같은 컨텍스트가 주어집니다.
"""

architect_prompt = """
Context:
#요구 사항:
{requirements}
#스케일:
{scale}
#특징:
{features}

AWS의 아키텍처를 기술적으로 자세히 설명하세요.
"""
context = architect_prompt.format(
    requirements="컴퓨터 광고 웹사이트", 
    scale="최대 초당 10,000건의 요청을 처리해야 합니다. 전 세계에서 사용할 수 있어야 합니다. 응답 속도가 빨라야 합니다.",
    features="제품을 설명하는 랜딩 페이지. 회사를 설명하는 소개 페이지."
)

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

잘 설계된 AWS 솔루션 아키텍처는 다음과 같이 구성됩니다:

1. Landing Page: 이 랜딩 페이지는 사용자가 컴퓨터 광고 웹사이트로 이동할 수 있도록 하며, 사용자가 원하는 제품이나 서비스를 쉽게 찾고 구매할 수 있도록 해야 합니다. 이 페이지는 사용자 친화적인 디자인으로 구성되어야 하며, 다양한 검색 기능을 제공하여 사용자가 원하는 정보를 빠르게 찾을 수 있도록 해야 합니다. 또한 이 페이지는 이미지 및 동영상과 같은 멀티미디어 콘텐츠를 포함하여 사용자의 관심을 끌기 위해 시각적으로
CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 6.58 s


In [79]:
%%time

instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

대규모 언어 모델(LLM) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 증가함에 따라, 대규모 언어 모델(LLM)은 훈련되지 않은 특정 작업에도 잘 일반화될 수 있으며, 프롬프트 엔지니어링 기법을 사용하면 유용하지만, 도메인이나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 할 때가 있습니다. 이러한 파운데이션 모델은 프롬프트 엔지니어링 기법을 통해 잘 작동할 수 있지만, 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하
CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 6.65 s


### Applying LangChain

In [97]:
llm.model_kwargs = params
print(llm(payload['inputs']))

외부 웹 애플리케이션은 JavaScript로 작성되며 Amazon Web Services(AWS)에서 호스팅됩니다. 이 애플리케이션은 분당 평균 500건 이상의 요청과 초당 최대 3000건에 달하는 요청을 처리할 수 있습니다. 모바일 웹사이트와 데스크톱 버전 모두 자바스크립트로 구축되었으며, AWS는 이러한 애플리케이션을 위한 아키텍처를 제공합니다. 스케일링 측면에서 애플리케이션은 초당 최대 3000개의 요청을 처리할 수 있도록 설계되었지만, AWS는 애플리케이션이 원활하고 효율적으로


In [85]:
from langchain.prompts import PromptTemplate

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt,
)

context = architect_prompt.format(
    requirements="자바스크립트로 작성된 외부 웹 애플리케이션, 글로벌 배포",
    scale="분당 평균 500건의 요청, 초당 최대 3000건의 요청까지 이벤트 확장",
    features="모바일 웹사이트, 데스크톱 버전, 자바스크립트"
)
print(context)


Context:
#요구 사항:
자바스크립트로 작성된 외부 웹 애플리케이션, 글로벌 배포
#스케일:
분당 평균 500건의 요청, 초당 최대 3000건의 요청까지 이벤트 확장
#특징:
모바일 웹사이트, 데스크톱 버전, 자바스크립트

AWS의 아키텍처를 기술적으로 자세히 설명하세요.



In [88]:
%%time
payload = get_payload(instruction, context, params)
print(llm(payload['inputs']))

외부 웹 애플리케이션은 JavaScript로 작성되며 Amazon Web Services(AWS)에서 호스팅됩니다. 이 애플리케이션은 분당 평균 500건 이상의 요청과 초당 최대 3000건에 달하는 요청을 처리할 수 있습니다. 모바일 웹사이트와 데스크톱 버전 모두 자바스크립트로 구축되었으며, AWS는 이러한 애플리케이션을 위한 아키텍처를 제공합니다. 스케일링 측면에서 애플리케이션은 초당 최대 3000개의 요청을 처리할 수 있도록 설계되었지만, AWS는 애플리케이션이 원활하고 효율적으로
CPU times: user 14 ms, sys: 749 µs, total: 14.7 ms
Wall time: 6.54 s


In [94]:
topic_recommender_prompt = "{topic}에 대한 블로그 게시물에 작성할 내용을 {number}개만큼 나열합니다."

recommend_topic_prompt = PromptTemplate(
    template=topic_recommender_prompt,
    input_variables=['topic', 'number']
)

results = llm(recommend_topic_prompt.format(topic="머신 러닝", number=5))
print(results)

1. 머신러닝이란 무엇인가요?2. 머신러닝의 종류는 무엇이 있나요?3. 머신러닝에서 사용되는 알고리즘은 무엇인가요?4. 머신러닝과 기존 소프트웨어 개발 프로세스와의 차이점은 무엇인가요?5. 머신러닝 프로젝트를 성공적으로 완료하기 위해 알아야 할 사항은 무엇인가요?​A. 1. 머신러닝이란 무엇인가요?- 머신러닝(Machine Learning)은 컴퓨터 시스템이 데이터로부터 학습하고, 예측이나 결정을 내릴 수 있도록 하는 인공 지능의 하위 집합입니다.- 머신러닝은 컴퓨터가 경험을 통해 자동


In [90]:
instruction = "다음 질문에 답변해 주세요."
question = "Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 무엇인가요?"
payload = get_payload(instruction, question, params)

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])
#payload = get_payload(instruction, context, params)

Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스 유형은 AWS CLI 또는 GCP 콘솔과 같은 원격 인터페이스를 통해 액세스할 수 있는 Amazon Elastic Beanstalk(EC2) 인스턴스입니다.


### With Context
- 추가 컨텍스트 or few-shot 제공

In [71]:
instruction = "위 컨텍스트 내용에 기반해서 질문에 답변해 주세요."
context = "관리형 스팟 훈련은 Amazon SageMaker에서 지원되는 모든 인스턴스에서 사용할 수 있고, 현재 Amazon SageMaker를 사용할 수 있는 모든 AWS 리전에서 지원됩니다."

question = "Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 무엇인가요?"
prompt = f"{context}\n\n{question}"

In [72]:
payload = get_payload(instruction, question, params)

response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

죄송하지만 Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 특정 인스턴스에 대한 정보가 없습니다. 하지만 AWS 관리형 스팟은 Amazon SageMaker에서 실행되며, Amazon SageMaker의 기본 제공 기능인 Amazon EC2 인스턴스와 함께 사용할 수 있다는 점을 언급하고 싶습니다. 따라서 Amazon SageMaker에서 관리형 스팟을 사용하여 훈련 프로세스를 자동화하고 비용 효율적인 방식


### Apply LangChain


In [75]:
#prompt = "What is Amazon SageMaker's advantages for Data Scientists? Please summarize in 100 words"
llm.model_kwargs = parameters
result = llm(prompt)
print(result)

Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 Amazon SageMaker에서 지원하는 모든 인스턴스입니다.Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 어디에서 찾을 수 있나요?Amazon SageMaker에서 관리형 스팟 훈련 기능을 사용할 수 있는 인스턴스는 Amazon SageMaker에서 지원하는 모든 인스턴스입니다


In [104]:
# from lib_ko import EmbeddingAmazonApiGateway
# emb = EmbeddingAmazonApiGateway(api_url=EMB_URL)

# prompt = "세이지메이커는 뭐야?"
# result = emb.embed_query(prompt)
# print(result[0:5])

In [105]:
# #testing our embeddings modelb
# emb_results = emb.embed_documents([prompt])
# print(emb_results[0][:5])
